In [1]:
# Import libraries and dependencies

import numpy as np
import pandas as pd
import time
from finsymbols import symbols

import api_calls

In [2]:
sp500 = symbols.get_sp500_symbols()
listings = api_calls.call_listings()

# Create a DataFrame from the listings data, with the first row as the column names
all_stocks = pd.DataFrame(listings[1:], columns=listings[0])

In [3]:
# Get the symbols from the S&P 500
sp500_symbols = [sp500[i]['symbol'] for i in range(len(sp500))]

# Remove \n from the symbols
sp500_symbols = [sp500_symbols[i].replace('\n', '') for i in range(len(sp500_symbols))]

# Replace "BF.B" with "BF-B"
sp500_symbols = [sp500_symbols[i].replace('BF.B', 'BF-B') for i in range(len(sp500_symbols))]

# Display sample data
sp500_symbols

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ATVI',
 'ADM',
 'ADBE',
 'ADP',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BALL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'WRB',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF-B',
 'BG',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA'

In [4]:
# Remove all ETFs from the DataFrame
all_stocks = all_stocks[all_stocks['assetType'] != 'ETF']

# Remove all stocks that are not on the NYSE or NASDAQ
all_stocks = all_stocks[(all_stocks['exchange'] == 'NYSE') | (all_stocks['exchange'] == 'NASDAQ')]

# Create a list with the symbols from all stocks
all_stocks_symbols = all_stocks['symbol'].tolist()

all_stocks_symbols

['A',
 'AA',
 'AAC',
 'AAC-U',
 'AAC-WS',
 'AACG',
 'AACI',
 'AACIU',
 'AACIW',
 'AADI',
 'AAIC',
 'AAIC-P-B',
 'AAIC-P-C',
 'AAIN',
 'AAL',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAT',
 'AB',
 'ABB',
 'ABBV',
 'ABC',
 'ABCB',
 'ABCL',
 'ABCM',
 'ABEO',
 'ABEV',
 'ABG',
 'ABIO',
 'ABM',
 'ABNB',
 'ABOS',
 'ABR',
 'ABR-P-D',
 'ABR-P-E',
 'ABR-P-F',
 'ABSI',
 'ABST',
 'ABT',
 'ABUS',
 'ABVC',
 'AC',
 'ACA',
 'ACAB',
 'ACABU',
 'ACABW',
 'ACAC',
 'ACACU',
 'ACACW',
 'ACAD',
 'ACAH',
 'ACAHU',
 'ACAHW',
 'ACAX',
 'ACAXR',
 'ACAXU',
 'ACAXW',
 'ACB',
 'ACBA',
 'ACBAU',
 'ACBAW',
 'ACCD',
 'ACCO',
 'ACDC',
 'ACDCW',
 'ACEL',
 'ACER',
 'ACET',
 'ACGL',
 'ACGLN',
 'ACGLO',
 'ACGN',
 'ACHC',
 'ACHL',
 'ACHR',
 'ACHV',
 'ACI',
 'ACIU',
 'ACIW',
 'ACLS',
 'ACLX',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACNT',
 'ACON',
 'ACONW',
 'ACOR',
 'ACP-P-A',
 'ACR',
 'ACR-P-C',
 'ACR-P-D',
 'ACRE',
 'ACRO',
 'ACRO-U',
 'ACRO-WS',
 'ACRS',
 'ACRV',
 'ACRX',
 'ACST',
 'ACT',
 'ACTG',
 'ACVA',
 'AC

In [5]:
# Create a function to calculate the 100-day return for stocks in the list
def get_returns(symbols):
    # Create an empty DataFrame with symbol and return columns
    df = pd.DataFrame(columns=['symbol', 'stock_return_100', 'market_return_100'])

    # Find the 100-day return for the market
    market_return_100 = get_return('VTI')

    for i, symbol in enumerate(symbols):
        print(symbol)
        # Calculate the 100-day return
        stock_return_100 = get_return(symbol)

        # Add the symbol, stock return, and market return to the DataFrame using pd.concat
        df = pd.concat([df, pd.DataFrame({'symbol': symbol, 'stock_return_100': stock_return_100, 'market_return_100': market_return_100}, index=[0])], axis=0)

        # If the index is at 1200, pause for 60 seconds to avoid the API limit
        if i % 1150 == 0:
            time.sleep(60)

    return df

# Create a function to calculate the return for a stock over the last 100 days
def get_return(symbol):
    # Call the alpha vantage api to get the data
    stock_data = api_calls.call_daily(symbol)

    # Return None if there is no data or less than 100 days of data
    try:
        if len(stock_data['Time Series (Daily)']) < 100:
            return None
    except:
        return None

    # Get the adjusted closing prices
    stock_data_df = pd.DataFrame(stock_data['Time Series (Daily)']).T

    # Calculate the return from the first day to the last day
    return_100 = (float(stock_data_df['5. adjusted close'][0]) - float(stock_data_df['5. adjusted close'][-1])) / float(stock_data_df['5. adjusted close'][-1])

    return return_100

In [6]:
# Create an empty DataFrame with symbol and return columns
df = pd.DataFrame(columns=['symbol', 'stock_return_100', 'market_return_100'])

# Indicate which symbols to use
symbols = all_stocks_symbols

In [7]:
# Loop through the symbols and get the company overview data, calculating the 100-day return for each company and saving it to a DataFrame

df = get_returns(symbols)

A
AA
AAC
AAC-U
AAC-WS
AACG
AACI
AACIU
AACIW
AADI
AAIC
AAIC-P-B
AAIC-P-C
AAIN
AAL
AAME
AAN
AAOI
AAON
AAP
AAPL
AAT
AB
ABB
ABBV
ABC
ABCB
ABCL
ABCM
ABEO
ABEV
ABG
ABIO
ABM
ABNB
ABOS
ABR
ABR-P-D
ABR-P-E
ABR-P-F
ABSI
ABST
ABT
ABUS
ABVC
AC
ACA
ACAB
ACABU
ACABW
ACAC
ACACU
ACACW
ACAD
ACAH
ACAHU
ACAHW
ACAX
ACAXR
ACAXU
ACAXW
ACB
ACBA
ACBAU
ACBAW
ACCD
ACCO
ACDC
ACDCW
ACEL
ACER
ACET
ACGL
ACGLN
ACGLO
ACGN
ACHC
ACHL
ACHR
ACHV
ACI
ACIU
ACIW
ACLS
ACLX
ACM
ACMR
ACN
ACNB
ACNT
ACON
ACONW
ACOR
ACP-P-A
ACR
ACR-P-C
ACR-P-D
ACRE
ACRO
ACRO-U
ACRO-WS
ACRS
ACRV
ACRX
ACST
ACT
ACTG
ACVA
ACXP
ADAG
ADAL
ADALU
ADALW
ADAP
ADBE
ADC
ADC-P-A
ADCT
ADD
ADEA
ADER
ADERU
ADERW
ADES
ADI
ADIL
ADILW
ADM
ADMA
ADMP
ADN
ADNT
ADNWW
ADOC
ADOCR
ADOCW
ADP
ADPT
ADSE
ADSEW
ADSK
ADT
ADTH
ADTHW
ADTN
ADTX
ADUS
ADV
ADVM
ADXN
AEAE
AEAEU
AEAEW
AEE
AEFC
AEG
AEHA
AEHL
AEHR
AEI
AEIS
AEL
AEL-P-A
AEL-P-B
AEM
AEMD
AENZ
AEO
AEP
AEPPZ
AER
AES
AESC
AESI
AEVA
AEVA-WS
AEY
AEYE
AEZS
AFAR
AFARU
AFARW
AFBI
AFCG
AFG
AFGB
AFGC
AFGD
AFGE
AFIB
AFL
AFMD
AFRI
AFRI

In [8]:
df

,symbol,stock_return_100,market_return_100
0,A,0.011998,0.093607
0,AA,-0.006511,0.093607
0,AAC,0.032516,0.093607
0,AAC-U,0.050050,0.093607
0,AAC-WS,16.600000,0.093607
...,...,...,...
0,ZXZZT,0.000000,0.093607
0,ZYME,0.250000,0.093607
0,ZYNE,-0.388165,0.093607
0,ZYRX,-0.529412,0.093607


In [8]:
print(api_calls.call_daily('A'))

{'Meta Data': {'1. Information': 'Daily Time Series with Splits and Dividend Events', '2. Symbol': 'A', '3. Last Refreshed': '2023-04-04', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2023-04-04': {'1. open': '138.25', '2. high': '139.11', '3. low': '137.53', '4. close': '138.21', '5. adjusted close': '138.21', '6. volume': '1778667', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2023-04-03': {'1. open': '137.42', '2. high': '138.09', '3. low': '136.005', '4. close': '137.86', '5. adjusted close': '137.86', '6. volume': '1551755', '7. dividend amount': '0.2250', '8. split coefficient': '1.0'}, '2023-03-31': {'1. open': '135.4', '2. high': '138.73', '3. low': '134.86', '4. close': '138.34', '5. adjusted close': '138.114584495057', '6. volume': '2579848', '7. dividend amount': '0.0000', '8. split coefficient': '1.0'}, '2023-03-30': {'1. open': '135.27', '2. high': '135.54', '3. low': '133.32', '4. close': '134.49', '5. adjusted c

In [9]:
# Print the companies that have None for the 100-day return
df[df['stock_return_100'].isnull()]

,symbol,stock_return_100,market_return_100
0,ACRV,NaN,0.093607
0,AESI,NaN,0.093607
0,AIXI,NaN,0.093607
0,ALB,NaN,0.093607
0,ALBT,NaN,0.093607
...,...,...,...
0,WAVSW,NaN,0.093607
0,YGF,NaN,0.093607
0,YS,NaN,0.093607
0,ZJYL,NaN,0.093607


In [ ]:
#Rerun the 100-day return data for the companies that have None for the 100-day return
for symbol in df[df['stock_return_100'].isnull()]['symbol']:
    # Calculate the 100-day return
    stock_return_100 = get_return(symbol)

    # Replace the None value with the new return
    df.loc[df['symbol'] == symbol, 'stock_return_100'] = stock_return_100

In [10]:
df.to_pickle('progress.csv')

In [24]:
df = pd.read_pickle('progress.csv')

In [ ]:
# Create a function to calculate the news sentiment for stocks in the list
def get_sentiment(symbols):
    # Create an empty DataFrame with symbol and sentiment columns
    df = pd.DataFrame(columns=['symbol', 'sentiment'])

    for i, symbol in enumerate(symbols):
        # Call the alpha vantage api to get the news data
        news = api_calls.call_news(symbol)

        # If there is no news data, return None
        try:
            if news['items'] == '0':
                df = pd.concat([df, pd.DataFrame({'symbol': symbol, 'sentiment': None}, index=[0])], axis=0)
                continue
        except:
            df = pd.concat([df, pd.DataFrame({'symbol': symbol, 'sentiment': None}, index=[0])], axis=0)
            continue

        # Create a Pandas DataFrame
        stock_news = pd.DataFrame(news['feed'])

        # Calculate the average sentiment
        sentiment = stock_news['overall_sentiment_score'].mean()

        # Add the symbol and sentiment to the DataFrame using pd.concat
        df = pd.concat([df, pd.DataFrame({'symbol': symbol, 'sentiment': sentiment}, index=[0])], axis=0)

        # If the index is at 1200, pause for 60 seconds to avoid the API limit
        if i % 1150 == 0:
            time.sleep(60)

    return df

In [25]:
# Append the news sentiment data to the company overview DataFrame
df = pd.merge(df, get_sentiment(symbols), on='symbol', how='left')

In [33]:
df

,symbol,stock_return_100,market_return_100,outperformed_market,sentiment
0,MMM,-0.153406,0.093607,False,0.098151
0,AOS,0.201845,0.093607,True,0.088535
0,ABT,0.038315,0.093607,False,0.174237
0,ABBV,0.092897,0.093607,False,0.155819
0,ACN,0.077087,0.093607,False,0.232305
...,...,...,...,...,...
0,YUM,0.088144,0.093607,False,0.159614
0,ZBRA,0.360675,0.093607,True,0.163402
0,ZBH,0.214013,0.093607,True,0.250696
0,ZION,-0.438187,0.093607,False,0.227761


In [45]:
# Print the companies that have None for the sentiment
df[df['sentiment'].isnull()]

,symbol,stock_return_100,market_return_100,outperformed_market,sentiment,beta
0,GOOGL,0.199267,0.093607,True,NaN,1.091
0,AJG,0.033975,0.093607,False,NaN,0.702
0,BRK.B,0.060490,0.093607,False,NaN,None
0,BF-B,-0.037810,0.093607,False,NaN,0.702
0,CE,0.164856,0.093607,True,NaN,1.304
0,CHD,0.233096,0.093607,True,NaN,0.453
0,CMCSA,0.212383,0.093607,True,NaN,1.001
0,EQR,0.017582,0.093607,False,NaN,0.815
0,EL,0.201014,0.093607,True,NaN,0.995
0,FOXA,0.199430,0.093607,True,NaN,0.869


In [44]:
# Rerun the sentiment data for the companies that have None for the sentiment
for symbol in df[df['sentiment'].isnull()]['symbol']:
    # Call the alpha vantage api to get the news data
    news = api_calls.call_news(symbol)

    # If there is no news data, return None
    try:
        if news['items'] == '0':
            df.loc[df['symbol'] == symbol, 'sentiment'] = None
            continue
    except:
        df.loc[df['symbol'] == symbol, 'sentiment'] = None
        continue

    # Create a Pandas DataFrame
    stock_news = pd.DataFrame(news['feed'])

    # Calculate the average sentiment
    sentiment = stock_news['overall_sentiment_score'].mean()

    # Replace the None with the sentiment
    df.loc[df['symbol'] == symbol, 'sentiment'] = sentiment

In [30]:
df.to_pickle('progress.csv')

In [35]:
df = pd.read_pickle('progress.csv')

In [ ]:
# Create a function to calculate the company overview data for stocks in the list
def get_overview(symbols):
    # Create an empty DataFrame with symbol and return columns
    df = pd.DataFrame(columns=['symbol', 'market_cap', 'ebitda', 'pe_ratio', 'peg_ratio', 'book_value', 'dividend_per_share', 'dividend_yield', 'eps', 'revenue_per_share', 'profit_margin', 'operating_margin', 'return_on_assets', 'return_on_equity', 'revenue', 'gross_profit', 'diluted_eps', 'quarterly_earnings_growth', 'quarterly_revenue_growth', 'trailing_pe', 'forward_pe', 'price_to_sales_ratio', 'price_to_book_ratio', 'ev_to_revenue', 'ev_to_ebitda', 'beta'])

    for i, symbol in enumerate(symbols):
        # Call the alpha vantage api to get the company overview data
        overview = api_calls.call_company_overview(symbol)

        # If there is no overview data, return None
        try:
            if overview['Symbol'] == '0':
                df = pd.concat([df, pd.DataFrame({'symbol': symbol, 'market_cap': None, 'ebitda': None, 'pe_ratio': None, 'peg_ratio': None, 'book_value': None, 'dividend_per_share': None, 'dividend_yield': None, 'eps': None, 'revenue_per_share': None, 'profit_margin': None, 'operating_margin': None, 'return_on_assets': None, 'return_on_equity': None, 'revenue': None, 'gross_profit': None, 'diluted_eps': None, 'quarterly_earnings_growth': None, 'quarterly_revenue_growth': None, 'trailing_pe': None, 'forward_pe': None, 'price_to_sales_ratio': None, 'price_to_book_ratio': None, 'ev_to_revenue': None, 'ev_to_ebitda': None, 'beta': None}, index=[0])], axis=0)
                continue
        except:
            df = pd.concat([df, pd.DataFrame({'symbol': symbol, 'market_cap': None, 'ebitda': None, 'pe_ratio': None, 'peg_ratio': None, 'book_value': None, 'dividend_per_share': None, 'dividend_yield': None, 'eps': None, 'revenue_per_share': None, 'profit_margin': None, 'operating_margin': None, 'return_on_assets': None, 'return_on_equity': None, 'revenue': None, 'gross_profit': None, 'diluted_eps': None, 'quarterly_earnings_growth': None, 'quarterly_revenue_growth': None, 'trailing_pe': None, 'forward_pe': None, 'price_to_sales_ratio': None, 'price_to_book_ratio': None, 'ev_to_revenue': None, 'ev_to_ebitda': None, 'beta': None}, index=[0])], axis=0)
            continue

        # Add the symbol and overview data to the DataFrame using pd.concat
        df = pd.concat([df, pd.DataFrame({'symbol': symbol, 'market_cap': overview['MarketCapitalization'], 'ebitda': overview['EBITDA'], 'pe_ratio': overview['PERatio'], 'peg_ratio': overview['PEGRatio'], 'book_value': overview['BookValue'], 'dividend_per_share': overview['DividendPerShare'], 'dividend_yield': overview['DividendYield'], 'eps': overview['EPS'], 'revenue_per_share': overview['RevenuePerShareTTM'], 'profit_margin': overview['ProfitMargin'], 'operating_margin': overview['OperatingMarginTTM'], 'return_on_assets': overview['ReturnOnAssetsTTM'], 'return_on_equity': overview['ReturnOnEquityTTM'], 'revenue': overview['RevenueTTM'], 'gross_profit': overview['GrossProfitTTM'], 'diluted_eps': overview['DilutedEPSTTM'], 'quarterly_earnings_growth': overview['QuarterlyEarningsGrowthYOY'], 'quarterly_revenue_growth': overview['QuarterlyRevenueGrowthYOY'], 'trailing_pe': overview['TrailingPE'], 'forward_pe': overview['ForwardPE'], 'price_to_sales_ratio': overview['PriceToSalesRatioTTM'], 'price_to_book_ratio': overview['PriceToBookRatio'], 'ev_to_revenue': overview['EVToRevenue'], 'ev_to_ebitda': overview['EVToEBITDA'], 'beta': overview['Beta']}, index=[0])], axis=0)

        # If the index is at 1200, pause for 60 seconds to avoid the API limit
        if i % 1150 == 0:
            time.sleep(60)

    return df

In [60]:
# Append the company overview data to the DataFrame
df = pd.concat([df, get_overview(df['symbol'])], axis=1)

In [65]:
df

,symbol,stock_return_100,market_return_100,sentiment,beta,market_cap,ebitda,pe_ratio,peg_ratio,book_value,...,diluted_eps,quarterly_earnings_growth,quarterly_revenue_growth,trailing_pe,forward_pe,price_to_sales_ratio,price_to_book_ratio,ev_to_revenue,ev_to_ebitda,return on equity
0,MMM,-0.153406,0.093607,0.098151,0.963,57964929000,7424000000,10.26,2.314,26.8,...,10.19,-0.578,-0.062,10.26,11.92,1.946,4.95,2.377,8.52,0.388
0,AOS,0.201845,0.093607,0.088535,1.256,10390498000,290900000,45.6,1.812,11.56,...,1.51,-0.134,-0.06,45.6,18.12,2.276,5.15,2.331,12.14,0.132
0,ABT,0.038315,0.093607,0.174237,0.666,176558326000,12103000000,25.98,18.76,21.11,...,3.91,-0.472,-0.12,25.98,25.13,4.202,5.44,4.452,15.2,0.19
0,ABBV,0.092897,0.093607,0.155819,0.587,282332955000,31089000000,24.14,1.279,9.75,...,6.63,-0.388,0.016,24.14,14.39,4.989,18.23,6.04,13.8,0.724
0,ACN,0.077087,0.093607,0.232305,1.242,180662305000,10699595000,26.32,2.975,37.64,...,10.86,-0.059,0.051,26.32,25.58,2.878,8.69,3.043,18.53,0.309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,YUM,0.088144,0.093607,0.159614,0.984,37034480000,2328000000,28.93,2.661,-31.7,...,4.57,0.177,0.068,28.93,25.97,5.47,-,7.17,20.83,0
0,ZBRA,0.360675,0.093607,0.163402,1.663,16346697000,1140000000,36.11,1.633,53.12,...,8.63,0.009,0.025,36.11,13.95,2.256,5.08,2.744,14.61,0.162
0,ZBH,0.214013,0.093607,0.250696,1.005,27010548000,2246400000,92.46,1.078,57.51,...,1.38,0.331,0.027,92.46,18.28,3.309,2.198,4.083,20.87,0.0236
0,ZION,-0.438187,0.093607,0.227761,1.171,4123444000,None,4.677,0.538,29.95,...,5.79,0.369,0.156,4.677,6.84,2.404,1.671,-,-,0.147


In [64]:
# Print the companies that have None for the beta
df[df['market_cap'].isnull()]

,symbol,stock_return_100,market_return_100,sentiment,beta,market_cap,ebitda,pe_ratio,peg_ratio,book_value,...,diluted_eps,quarterly_earnings_growth,quarterly_revenue_growth,trailing_pe,forward_pe,price_to_sales_ratio,price_to_book_ratio,ev_to_revenue,ev_to_ebitda,return on equity
0,BRK.B,0.06049,0.093607,NaN,NaN,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
# Rerun the api call for the companies that have None for the market cap
for symbol in df[df['beta'].isnull()]['symbol']:
    # Call the alpha vantage api to get the company overview data
    company_overview = api_calls.call_company_overview(symbol)

    # If there is no company overview data, return None
    try:
        if company_overview['MarketCapitalization'] == 'N/A':
            df.loc[df['symbol'] == symbol, 'market_cap'] = None
            continue
    except:
        df.loc[df['symbol'] == symbol, 'market_cap'] = None
        continue

    # Add the market cap, ebitda, pe ratio, peg ratio, book value, dividend per share, dividend yield, eps, revenue per share, profit margin, operating margin, return on assets, return on equity, revenue, gross profit, diluted eps, quarterly earnings growth, quarterly revenue growth, trailing pe, forward pe, price to sales ratio, price to book ratio, ev to revenue, ev to ebitda, beta to the DataFrame
    df.loc[df['symbol'] == symbol, 'market_cap'] = company_overview['MarketCapitalization']
    df.loc[df['symbol'] == symbol, 'ebitda'] = company_overview['EBITDA']
    df.loc[df['symbol'] == symbol, 'pe_ratio'] = company_overview['PERatio']
    df.loc[df['symbol'] == symbol, 'peg_ratio'] = company_overview['PEGRatio']
    df.loc[df['symbol'] == symbol, 'book_value'] = company_overview['BookValue']
    df.loc[df['symbol'] == symbol, 'dividend_per_share'] = company_overview['DividendPerShare']
    df.loc[df['symbol'] == symbol, 'dividend_yield'] = company_overview['DividendYield']
    df.loc[df['symbol'] == symbol, 'eps'] = company_overview['EPS']
    df.loc[df['symbol'] == symbol, 'revenue_per_share'] = company_overview['RevenuePerShareTTM']
    df.loc[df['symbol'] == symbol, 'profit_margin'] = company_overview['ProfitMargin']
    df.loc[df['symbol'] == symbol, 'operating_margin'] = company_overview['OperatingMarginTTM']
    df.loc[df['symbol'] == symbol, 'return_on_assets'] = company_overview['ReturnOnAssetsTTM']
    df.loc[df['symbol'] == symbol, 'return_on_equity'] = company_overview['ReturnOnEquityTTM']
    df.loc[df['symbol'] == symbol, 'revenue'] = company_overview['RevenueTTM']
    df.loc[df['symbol'] == symbol, 'gross_profit'] = company_overview['GrossProfitTTM']
    df.loc[df['symbol'] == symbol, 'diluted_eps'] = company_overview['DilutedEPSTTM']
    df.loc[df['symbol'] == symbol, 'quarterly_earnings_growth'] = company_overview['QuarterlyEarningsGrowthYOY']
    df.loc[df['symbol'] == symbol, 'quarterly_revenue_growth'] = company_overview['QuarterlyRevenueGrowthYOY']
    df.loc[df['symbol'] == symbol, 'trailing_pe'] = company_overview['TrailingPE']
    df.loc[df['symbol'] == symbol, 'forward_pe'] = company_overview['ForwardPE']
    df.loc[df['symbol'] == symbol, 'price_to_sales_ratio'] = company_overview['PriceToSalesRatioTTM']
    df.loc[df['symbol'] == symbol, 'price_to_book_ratio'] = company_overview['PriceToBookRatio']
    df.loc[df['symbol'] == symbol, 'ev_to_revenue'] = company_overview['EVToRevenue']
    df.loc[df['symbol'] == symbol, 'ev_to_ebitda'] = company_overview['EVToEBITDA']
    df.loc[df['symbol'] == symbol, 'beta'] = company_overview['Beta']

In [66]:
df.to_pickle('progress.csv')

In [3]:
df = pd.read_pickle('progress.csv')

In [8]:
# drop the return on equity column
df = df.drop(columns=['return on equity'])

In [9]:
df.to_csv('stocks.csv')

In [6]:
all_stocks = api_calls.call_listings()
all_stocks

[['symbol',
  'name',
  'exchange',
  'assetType',
  'ipoDate',
  'delistingDate',
  'status'],
 ['A',
  'Agilent Technologies Inc',
  'NYSE',
  'Stock',
  '1999-11-18',
  'null',
  'Active'],
 ['AA', 'Alcoa Corp', 'NYSE', 'Stock', '2016-10-18', 'null', 'Active'],
 ['AAA',
  'AXS FIRST PRIORITY CLO BOND ETF ',
  'NYSE ARCA',
  'ETF',
  '2020-09-09',
  'null',
  'Active'],
 ['AAAU',
  'Goldman Sachs Physical Gold ETF',
  'BATS',
  'ETF',
  '2018-08-15',
  'null',
  'Active'],
 ['AAC',
  'Ares Acquisition Corporation - Class A',
  'NYSE',
  'Stock',
  '2021-03-25',
  'null',
  'Active'],
 ['AAC-U',
  'Ares Acquisition Corporation - Units (1 Ord Share Class A & 1/5 War)',
  'NYSE',
  'Stock',
  '2021-02-02',
  'null',
  'Active'],
 ['AAC-WS',
  'Ares Acquisition Corporation - Warrants (01/01/9999)',
  'NYSE',
  'Stock',
  '2021-03-25',
  'null',
  'Active'],
 ['AACG',
  'ATA Creativity Global',
  'NASDAQ',
  'Stock',
  '2008-01-29',
  'null',
  'Active'],
 ['AACI',
  'Armada Acquisition C

In [5]:
# Find all stocks in the

11662

In [30]:
api_calls.call_news('ABR')

{'items': '200',
 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish',
 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.',
 'feed': [{'title': 'Arbor Realty Trust  ( ABR )  Stock Sinks As Market Gains: What You Should Know',
   'url': 'https://www.zacks.com/stock/news/2074000/arbor-realty-trust-abr-stock-sinks-as-market-gains-what-you-should-know',
   'time_published': '20230403T215018',
   'authors': ['Zacks Investment Research'],
   'summary': 'Arbor Realty Trust (ABR) closed the most recent trading day at $11.46, moving -0.26% from the previous trading session.',
   'banner_image': 'https://staticx-tuner.zacks.com/images/default_article_images/default182.jpg',
   'source': 'Zacks Commentary',
   'category_within_source': 'n/a',
   'source_domain': 'www.zacks.com',
   'topics': [{'topic': 'Earnings', 'relevance_score': '0.